In [1]:
import pandas as pd

df_usagers = pd.read_csv("cleaned_files/usagers_clean.csv", sep=";", dtype=str)
df_vehicules= pd.read_csv("cleaned_files/véhicules_clean.csv", sep=";",dtype=str)
df_caracteristiques = pd.read_csv("cleaned_files/caractéristiques_clean.csv", sep=";", dtype=str)
df_lieux = pd.read_csv("csv/lieux_apres_pivot.csv", sep=";", dtype=str)

In [2]:

df = df_usagers.merge(df_vehicules,on=['Accident_Id','Numéro du véhicule'])

In [10]:
df.shape

(620105, 25)

VELOS ET VAE (LES CYCLISTES, N'INCLUT PAS LES PIETONS HEURTES PAR DES VELOS)

In [12]:
catv = ["01 - Bicyclette", "80 - VAE"]

# Filter DataFrame by list of user

filtered_df = df[df['Catégorie du véhicule'].isin(catv) & df['Catégorie usager'].isin(["1 - Conducteur"])]

s_acc = filtered_df["Accident_Id"]

list_acc = s_acc.to_list()

accidents_cyclistes = df[df['Accident_Id'].isin(list_acc)]
#print(filtered_acc_df)

In [13]:
accidents_cyclistes.shape

(53787, 25)

In [14]:
accidents_cyclistes.to_csv("csv/accidents_cyclistes_sans_pivot.csv", index=False)

ON FAIT LE PIVOT


In [15]:
df = accidents_cyclistes
df=df.sort_values(['Accident_Id','Catégorie du véhicule',"Catégorie usager"],ascending=True)

# Ajouter un index par groupe ID
df['row'] = df.groupby('Accident_Id').cumcount() + 1

# Transformer les lignes en colonnes
df_pivot_md = df.pivot(index='Accident_Id', columns='row', values=['Catégorie du véhicule','Catégorie usager', 'Gravité des blessures', 'Sexe', 'Année de naissance', 'Motif du déplacement', 'Equipement de sécurité 1'])

# Aplatir l'index des colonnes
df_pivot_md.columns = [f"{col[0]}{col[1]}" for col in df_pivot_md.columns]

# Réinitialiser l'index
df_pivot_md.reset_index(inplace=True)

In [16]:
accident = df_caracteristiques.merge(df_lieux,on = 'Accident_Id')
df_complete = df_pivot_md.merge(accident,on='Accident_Id')

SAUVEGARDE DU DATASET EN CSV

In [17]:
df_complete.to_csv('datasets/table_complete_accidents_cyclistes_avec_pivot_véhicules.csv',sep=";",index=False)